In [ ]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-qwpc4ti7/farm-haystack_c2be091b07e64540bff6701ec0af897f
  Resolved https://github.com/deepset-ai/haystack.git to commit 3319ef6d1c8f0b8a4d80d0f531c6ea0fd1c02e6e
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with sta

  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-qwpc4ti7/farm-haystack_c2be091b07e64540bff6701ec0af897f


In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
from haystack.utils import launch_es

launch_es()

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time
time.sleep(30)

In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/My Drive')

Mounted at /content/drive/


In [ ]:
from haystack.utils import convert_files_to_docs

docs = convert_files_to_docs(dir_path= 'data/', split_paragraphs=True)
print(docs)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

INFO:haystack.utils.preprocessing:Converting data/preamble.txt
INFO:haystack.utils.preprocessing:Converting data/admission.txt
INFO:haystack.utils.preprocessing:Converting data/scope.txt
INFO:haystack.utils.preprocessing:Converting data/Course_plan.txt
INFO:haystack.utils.preprocessing:Converting data/Course_duration.txt
INFO:haystack.utils.preprocessing:Converting data/Course_and_credit_req.txt
INFO:haystack.utils.preprocessing:Converting data/external_programs.txt
INFO:haystack.utils.preprocessing:Converting data/Course_registration.txt
INFO:haystack.utils.preprocessing:Converting data/Attendance.txt
INFO:haystack.utils.preprocessing:Converting data/Assesment_and_evaluation.txt
INFO:haystack.utils.preprocessing:Converting data/Class_based_learning.txt
INFO:haystack.utils.preprocessing:Converting data/Questions_based_on_HOTS.txt
INFO:haystack.utils.preprocessing:Converting data/Eligibility.txt
INFO:haystack.utils.preprocessing:Converting data/CAL.txt
INFO:haystack.utils.preprocessing:

[<Document: {'content': 'Present day students are much different from the students of the past in many ways.\nToday, they like to make decisions on their own and plan their future by themselves.\nHowever, student aspirations on one hand and the demands of the work place on the other\nhave become highly diverse. Employers expect students to have multi-disciplinary competency,\nleadership skills, and be ICT (Information and Communication Technology) ready. The rigid,\ncohort system of learning, offers little flexibility to students in selecting the courses of their\nchoice and helps little in becoming a well-rounded personality.\nAs part of continuous improvement in providing quality education, during 2008, VIT has\ntaken the right step in this direction by introducing the Fully Flexible Credit System (FFCS™)\ninto its academic curriculum. Through this, the students can register for courses of their\nchoice altering at will, the pace of learning within the broad framework of an academic 

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [ ]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [ ]:
from haystack.nodes import TransformersReader
# reader = TransformersReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
prediction = pipe.run(
    query="When will you get debarred ?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 2/2 [00:38<00:00, 19.13s/ Batches]


In [ ]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'if the malpractice is proven', 'type': 'extractive', 'score': 0.5263640880584717, 'context': 'ted by the University and suitable punishment will be given,\nif the malpractice is proven. If a student indulges in malpractice, in any component of t', 'offsets_in_document': [{'start': 296, 'end': 324}], 'offsets_in_context': [{'start': 61, 'end': 89}], 'document_id': '39a13e9cc9fc5e132ad47f90d4b1eda5', 'meta': {'name': 'Academic_malpractice.txt'}}>,
             <Answer {'answer': 'if the student drops below the minimum attendance requirement', 'type': 'extractive', 'score': 0.48127245903015137, 'context': ' status of that exam\ncomponent as ‘Debarred’ if the student drops below the minimum attendance requirement. \nSuch display will be enabled one day prio', 'offsets_in_document': [{'start': 1195, 'end': 1256}], 'offsets_in_context': [{'start': 45, 'end': 106}], 'document_id': 'b43e92db3bd51bf7dfd71f30af3b2900', 'meta': {'name': 'Attendance.txt'}}>,
      

In [ ]:
from haystack.utils import print_answers
print_answers(prediction, details="all")


Query: When will you get debarred ?
Answers:
[   <Answer {'answer': 'if the malpractice is proven', 'type': 'extractive', 'score': 0.5263640880584717, 'context': 'ted by the University and suitable punishment will be given,\nif the malpractice is proven. If a student indulges in malpractice, in any component of t', 'offsets_in_document': [{'start': 296, 'end': 324}], 'offsets_in_context': [{'start': 61, 'end': 89}], 'document_id': '39a13e9cc9fc5e132ad47f90d4b1eda5', 'meta': {'name': 'Academic_malpractice.txt'}}>,
    <Answer {'answer': 'if the student drops below the minimum attendance requirement', 'type': 'extractive', 'score': 0.48127245903015137, 'context': ' status of that exam\ncomponent as ‘Debarred’ if the student drops below the minimum attendance requirement. \nSuch display will be enabled one day prio', 'offsets_in_document': [{'start': 1195, 'end': 1256}], 'offsets_in_context': [{'start': 45, 'end': 106}], 'document_id': 'b43e92db3bd51bf7dfd71f30af3b2900', 'meta': {'name':